In [2]:
# --- 라이브러리 임포트 (기존 코드 + 추가 라이브러리) ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# 경고 메시지 무시
warnings.filterwarnings('ignore')
# 시각화 설정
plt.rcParams['axes.unicode_minus'] = False 
sns.set_style('whitegrid')

# --- 데이터 불러오기 ---
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("데이터 로드 완료")
print(f"Train shape: {train.shape}, Test shape: {test.shape}\n")

데이터 로드 완료
Train shape: (6818, 12), Test shape: (1705, 11)



In [ ]:
# [전처리] (train, test 데이터에 동일하게 적용)
datasets = [train, test]
for df in datasets:
    # 1. Item_Fat_Content 값 통일
    df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'low fat':'Low Fat', 'LF':'Low Fat', 'reg':'Regular'})

    # 2. Item_Visibility의 0값을 결측치로 변경 (이후 평균값으로 대체)
    df['Item_Visibility'] = df['Item_Visibility'].replace(0, np.nan)

    # 3. 파생 변수 생성: Outlet_Age (성능 개선안에 사용)
    # (데이터 수집 시점이 2013년으로 보임)
    df['Outlet_Age'] = 2013 - df['Outlet_Establishment_Year']

In [ ]:
# 4. 결측치 채우기
# (주의: test의 결측치는 train의 통계값으로 채워야 함)

# Item_Weight: 상품(Item_Identifier)별 평균 무게로 채우기
item_avg_weight = train.groupby('Item_Identifier')['Item_Weight'].transform('mean')
train['Item_Weight'] = train['Item_Weight'].fillna(item_avg_weight)
test['Item_Weight'] = test['Item_Weight'].fillna(item_avg_weight)
# 그래도 남는 결측치는 전체 평균으로
train_global_avg_weight = train['Item_Weight'].mean()
train['Item_Weight'] = train['Item_Weight'].fillna(train_global_avg_weight)
test['Item_Weight'] = test['Item_Weight'].fillna(train_global_avg_weight)

# Outlet_Size: 매장 유형(Outlet_Type)별 최빈값으로 채우기
outlet_size_mode = train.groupby('Outlet_Type')['Outlet_Size'].apply(lambda x: x.mode()[0])
train['Outlet_Size'] = train.apply(
    lambda row: outlet_size_mode[row['Outlet_Type']] if pd.isnull(row['Outlet_Size']) else row['Outlet_Size'],
    axis=1
)
test['Outlet_Size'] = test.apply(
    lambda row: outlet_size_mode[row['Outlet_Type']] if pd.isnull(row['Outlet_Size']) else row['Outlet_Size'],
    axis=1
)

# Item_Visibility: 상품(Item_Identifier)별 평균 가시성으로 채우기
item_avg_visibility = train.groupby('Item_Identifier')['Item_Visibility'].transform('mean')
train['Item_Visibility'] = train['Item_Visibility'].fillna(item_avg_visibility)
test['Item_Visibility'] = test['Item_Visibility'].fillna(item_avg_visibility)
# 그래도 남는 결측치는 전체 평균으로
train_global_avg_visibility = train['Item_Visibility'].mean()
train['Item_Visibility'] = train['Item_Visibility'].fillna(train_global_avg_visibility)
test['Item_Visibility'] = test['Item_Visibility'].fillna(train_global_avg_visibility)

print("결측치 처리 및 파생 변수 생성 완료.")
print(f"Train 결측치 개수: {train.isnull().sum().sum()}")
print(f"Test 결측치 개수: {test.isnull().sum().sum()}\n")